### Analyzing car listing from a eBay Kleinanzeigen dataset. Data collected done by bot scraping.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

In [2]:
autos = pd.read_csv("autos.csv", encoding ="Latin-1")

In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [4]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


<ol>
<li>Price column needed to be converted into integer
</li>
<li>There are a few missing values in vehicle_Type, gearbox, model, fuel_type, not_repair_damage
</li>
</ol>

In [5]:
column = autos.columns

In [6]:
column = column.str.replace("yearOfRegistration", "registration_year")
column = column.str.replace("monthOfRegistration", "registration_month")
column = column.str.replace("notRepairedDamage", "unrepaired_damage")
column = column.str.replace("dateCreated", "ad_created")
print(column)


Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'registration_year', 'gearbox', 'powerPS', 'model',
       'odometer', 'registration_month', 'fuelType', 'brand',
       'unrepaired_damage', 'ad_created', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [7]:
def changeCase(str):
    res = [str[0].lower()]
    for character in str[1:]:
        if character in ('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
            res.append("_")
            res.append(character.lower())
        else:
            res.append(character)
    return "".join(res)

new_column = []
for c in column:
    sneakCase = changeCase(c)
    sneakCase= sneakCase.strip()
    new_column.append(sneakCase)
    
print(new_column)

autos.columns= new_column
autos.head(2)

['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest', 'vehicle_type', 'registration_year', 'gearbox', 'power_p_s', 'model', 'odometer', 'registration_month', 'fuel_type', 'brand', 'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code', 'last_seen']


,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08


<h> *I modified the names of columns from camelcase to sneakcase for readability*</h>

In [8]:
autos["odometer"] = autos["odometer"].str.replace(",", "")
autos["odometer"] = autos["odometer"].str.replace("km", "").astype(int)
autos.rename({"odometer": "odometer_km"}, axis = 1, inplace = True)



<ul>
<li> Number of pictures colunm doesnt contain any value </li>
<li> Price and odometer are numeric value stored as text</li>
<li> Horsepower cannot be zero, power_p_s column needed to be investigated</li>
</ul>

In [9]:
autos["price"] = autos["price"].str.replace(",", "")
autos["price"] = autos["price"].str.replace("$", "").astype(int)
autos.rename({"price": "price_US"}, axis = 1, inplace = True)


In [10]:
inspec_1 = autos["price_US"].describe()
inspec_2 = autos["odometer_km"].describe()

In [11]:
autos["price_US"].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price_US, dtype: int64

In [12]:
odometer_prop = autos["odometer_km"]
print(odometer_prop.value_counts().sort_index(ascending = True)
     )
print(autos["odometer_km"].describe())


5000        967
10000       264
20000       784
30000       789
40000       819
50000      1027
60000      1164
70000      1230
80000      1436
90000      1757
100000     2169
125000     5170
150000    32424
Name: odometer_km, dtype: int64
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


In [13]:
print(autos["price_US"].value_counts().sort_index(ascending = True).head()
     )

0    1421
1     156
2       3
3       1
5       2
Name: price_US, dtype: int64


In [14]:
# autos = autos[autos["price_US"].between(1, 351000)]
too_cheap = autos["price_US"] < 100
too_expensive = autos["price_US"] > 350000
autos.loc[too_cheap | too_expensive, "price_US"] = np.nan


In [15]:
autos["price_US"].describe()

count     48224.000000
mean       5930.371433
std        9078.372762
min         100.000000
25%        1250.000000
50%        3000.000000
75%        7499.000000
max      350000.000000
Name: price_US, dtype: float64

<h> Price_US column has a few outliers namely the values over 350000 and 0 usd. </h>

<li> <h> Value increases nicely to around 350,000 then spookily shoot up 10 mil USD, a little bit excessive for car sale on ebay. I removed 10 outliers ranged from 1mil to 10mil USD </h> </li>

In [16]:
(autos["date_crawled"].str[:10]
                     .value_counts(normalize = True, dropna = False)
                     .sort_index()
)

2016-03-05    0.02538
2016-03-06    0.01394
2016-03-07    0.03596
2016-03-08    0.03330
2016-03-09    0.03322
2016-03-10    0.03212
2016-03-11    0.03248
2016-03-12    0.03678
2016-03-13    0.01556
2016-03-14    0.03662
2016-03-15    0.03398
2016-03-16    0.02950
2016-03-17    0.03152
2016-03-18    0.01306
2016-03-19    0.03490
2016-03-20    0.03782
2016-03-21    0.03752
2016-03-22    0.03294
2016-03-23    0.03238
2016-03-24    0.02910
2016-03-25    0.03174
2016-03-26    0.03248
2016-03-27    0.03104
2016-03-28    0.03484
2016-03-29    0.03418
2016-03-30    0.03362
2016-03-31    0.03192
2016-04-01    0.03380
2016-04-02    0.03540
2016-04-03    0.03868
2016-04-04    0.03652
2016-04-05    0.01310
2016-04-06    0.00318
2016-04-07    0.00142
Name: date_crawled, dtype: float64

In [17]:
(autos["last_seen"]
     .str[:10]
     .value_counts(normalize = True, dropna =False)
     .sort_index()
)

2016-03-05    0.00108
2016-03-06    0.00442
2016-03-07    0.00536
2016-03-08    0.00760
2016-03-09    0.00986
2016-03-10    0.01076
2016-03-11    0.01252
2016-03-12    0.02382
2016-03-13    0.00898
2016-03-14    0.01280
2016-03-15    0.01588
2016-03-16    0.01644
2016-03-17    0.02792
2016-03-18    0.00742
2016-03-19    0.01574
2016-03-20    0.02070
2016-03-21    0.02074
2016-03-22    0.02158
2016-03-23    0.01858
2016-03-24    0.01956
2016-03-25    0.01920
2016-03-26    0.01696
2016-03-27    0.01602
2016-03-28    0.02086
2016-03-29    0.02234
2016-03-30    0.02484
2016-03-31    0.02384
2016-04-01    0.02310
2016-04-02    0.02490
2016-04-03    0.02536
2016-04-04    0.02462
2016-04-05    0.12428
2016-04-06    0.22100
2016-04-07    0.13092
Name: last_seen, dtype: float64

In [18]:
(autos["last_seen"]
     .str[:10]
     .value_counts(normalize = True, dropna = False)
     .sort_values(ascending = False)
)

2016-04-06    0.22100
2016-04-07    0.13092
2016-04-05    0.12428
2016-03-17    0.02792
2016-04-03    0.02536
2016-04-02    0.02490
2016-03-30    0.02484
2016-04-04    0.02462
2016-03-31    0.02384
2016-03-12    0.02382
2016-04-01    0.02310
2016-03-29    0.02234
2016-03-22    0.02158
2016-03-28    0.02086
2016-03-21    0.02074
2016-03-20    0.02070
2016-03-24    0.01956
2016-03-25    0.01920
2016-03-23    0.01858
2016-03-26    0.01696
2016-03-16    0.01644
2016-03-27    0.01602
2016-03-15    0.01588
2016-03-19    0.01574
2016-03-14    0.01280
2016-03-11    0.01252
2016-03-10    0.01076
2016-03-09    0.00986
2016-03-13    0.00898
2016-03-08    0.00760
2016-03-18    0.00742
2016-03-07    0.00536
2016-03-06    0.00442
2016-03-05    0.00108
Name: last_seen, dtype: float64

- It appeared that the bot crawled for 1 month from 05/3/2016 to 07/4/2017 

- The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.
- The last_seen_ad peaked at the last 3 days, this is due to the bot stop crawling not an increase in sales.

In [19]:
(autos["ad_created"]
        .str[:10]
        .value_counts(normalize = True, dropna = False)
        .sort_values(ascending = False)
)

2016-04-03    0.03892
2016-03-20    0.03786
2016-03-21    0.03772
2016-04-04    0.03688
2016-03-12    0.03662
2016-03-14    0.03522
2016-04-02    0.03508
2016-03-28    0.03496
2016-03-07    0.03474
2016-03-29    0.03414
2016-03-19    0.03384
2016-04-01    0.03380
2016-03-15    0.03374
2016-03-30    0.03344
2016-03-08    0.03334
2016-03-09    0.03324
2016-03-22    0.03280
2016-03-11    0.03278
2016-03-26    0.03256
2016-03-23    0.03218
2016-03-31    0.03192
2016-03-25    0.03188
2016-03-10    0.03186
2016-03-17    0.03120
2016-03-27    0.03090
2016-03-16    0.03000
2016-03-24    0.02908
2016-03-05    0.02304
2016-03-13    0.01692
2016-03-06    0.01512
               ...   
2016-02-25    0.00006
2016-02-18    0.00004
2016-02-20    0.00004
2016-02-05    0.00004
2016-01-10    0.00004
2016-02-24    0.00004
2016-02-02    0.00004
2016-02-14    0.00004
2016-02-26    0.00004
2016-02-09    0.00004
2016-01-22    0.00002
2016-01-16    0.00002
2016-01-07    0.00002
2015-12-30    0.00002
2015-06-11

- The majority of ad created within the first 1-2 months of listing date. 


In [20]:
autos["registration_year"].describe()

count    50000.000000
mean      2005.073280
std        105.712813
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Some registration days are quite odd. The min dated back to the 1000s, which car wasnt invented. And some dated to the future 9999.

In [21]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

0.03944

Outliers account for less than 4% in the registration year. We can remove it. 

In [22]:
autos = autos[autos["registration_year"].between(1900, 2016)]


In [23]:
autos["registration_year"].value_counts(normalize = True, dropna = False).head()

2000    0.069834
2005    0.062776
1999    0.062464
2004    0.056988
2003    0.056779
Name: registration_year, dtype: float64

Most cars were registered in the 2000s year

In [24]:
brand_count = autos["brand"].value_counts(normalize = True)
popular_brand = brand_count[brand_count > 0.05].index
print(popular_brand)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [25]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price_US', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_p_s', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [31]:
mean_price_brand  = {}

for b in popular_brand:
    brand_only = autos[autos["brand"] == b]
    mean_price = brand_only["price_US"].mean()
    mean_price_brand[b] = int(mean_price)
print(mean_price_brand)

{'opel': 3005, 'bmw': 8381, 'mercedes_benz': 8672, 'audi': 9380, 'ford': 3779, 'volkswagen': 5436}


- The most expensive cars' brands are: audi, mercedes and bmw
- The volkswagen is the most popular among popular cars since it has its price between the most expensive and the cheapest brands in popular_car. 

In [33]:
mean_mileage_brand ={}

for b in popular_brand:
    brand_only =brand_only = autos[autos["brand"] == b]
    mean_mileage = brand_only["odometer_km"].mean()
    mean_mileage_brand[b] = int(mean_mileage)
    
print(mean_mileage_brand)


{'opel': 129227, 'bmw': 132434, 'mercedes_benz': 130860, 'audi': 129287, 'ford': 124046, 'volkswagen': 128730}


In [36]:
mean_mileage = pd.Series(mean_mileage_brand).sort_values(ascending=False)
mean_prices = pd.Series(mean_price_brand).sort_values(ascending=False)

In [39]:
brand_info = pd.DataFrame(mean_mileage, columns = ["mean_mileage"])
brand_info["mean_price"] = mean_prices
print(brand_info)

               mean_mileage  mean_price
bmw                  132434        8381
mercedes_benz        130860        8672
audi                 129287        9380
opel                 129227        3005
volkswagen           128730        5436
ford                 124046        3779


There is a slight trend that more expensive cars have more mileage while less expensive cars have less mileage. 